In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
import os
import requests
import time

import ccxt
import matplotlib.pyplot as plt
import pandas as pd

import core.config.config_ as cconconf
import core.statistics as costatis
import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import helpers.hs3 as hs3
import helpers.hsecrets as hsecret
import im_v2.ccxt.data.client as icdcl
import im_v2.ccxt.data.extract.exchange_class as imvcdeexcl

/app/helpers/hparquet.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-a726fafc-c73f-4c3b-ac7a-11579bab1443.json'
INFO  # Git
    branch_name='master'
    hash='fe1fc6acb'
    # Last commits:
      * fe1fc6acb Toma Jordania Cm task1873 download of trade data (#1886)                        (  27 hours ago) Thu May 12 15:59:45 2022  (HEAD -> master, origin/master, origin/HEAD)
      * cb933ec8a Max Sergeychikov CMTask1706 QA for raw data (#1784)                                (    2 days ago) Wed May 11 21:56:33 2022  (origin/CmTask1889_Abstract_periodical_rt_download_script, origin/CMTask1876_sync_csv_with_CCXT)
      * a3fc660cc Juraj Smeriga CmTask1142_Standardize_DB_table_timestamp_names_2 (#1893)         (    2 days ago) Wed May 11 21:05:19 2022           
# Machine info
    system=Linux
    node name=3b01b48470d2
    release=5.13.0-1022-aws
    version=#24~20.04.1-Ubuntu SMP Thu Apr 7 22:10:15 UTC 2022
    machine=x86_64
    processor=x86_

In [4]:
def get_cmtask1866_config_ccxt() -> cconconf.Config:
    """
    Get task1866-specific config.
    """
    config = cconconf.Config()
    # Load parameters.
    config.add_subconfig("load")
    config["load"]["aws_profile"] = "ck"
    #
    s3_bucket_path = hs3.get_s3_bucket_path(config["load"]["aws_profile"])
    s3_path = "s3://cryptokaizen-data/historical"
    config["load"]["data_dir"] = s3_path
    # Data parameters.
    config.add_subconfig("data")
    config["data"]["vendor"] = "CCXT"
    config["data"]["data_snapshot"] = "latest"
    config["data"]["version"] = "v3"
    config["data"]["resample_1min"] = True
    config["data"]["partition_mode"] = "by_year_month"
    config["data"]["start_ts"] = None
    config["data"]["end_ts"] = None
    config["data"]["columns"] = None
    config["data"]["filter_data_mode"] = "assert"
    # Column names.
    config.add_subconfig("column_names")
    config["column_names"]["full_symbol"] = "full_symbol"
    config["column_names"]["close_price"] = "close"
    return config

In [5]:
config = get_cmtask1866_config_ccxt()
print(config)

load:
  aws_profile: ck
  data_dir: s3://cryptokaizen-data/historical
data:
  vendor: CCXT
  data_snapshot: latest
  version: v3
  resample_1min: True
  partition_mode: by_year_month
  start_ts: None
  end_ts: None
  columns: None
  filter_data_mode: assert
column_names:
  full_symbol: full_symbol
  close_price: close


In [6]:
def _get_qa_stats(data: pd.DataFrame, config: cconconf.Config) -> pd.DataFrame:
    """
    Get quality assurance stats per full symbol in data.
    """
    res_stats = []
    for full_symbol, symbol_data in data.groupby(
        config["column_names"]["full_symbol"]
    ):
        # Compute stats for a full symbol.
        symbol_stats = pd.Series(dtype="object", name=full_symbol)
        symbol_stats["min_timestamp"] = symbol_data.index.min()
        symbol_stats["max_timestamp"] = symbol_data.index.max()
        symbol_stats["NaNs %"] = 100 * (
            costatis.compute_frac_nan(
                symbol_data[config["column_names"]["close_price"]]
            )
        )
        symbol_stats["volume=0 %"] = 100 * (
            symbol_data[symbol_data["volume"] == 0].shape[0]
            / symbol_data.shape[0]
        )
        symbol_stats["bad data %"] = symbol_stats["NaNs %"] + symbol_stats["volume=0 %"]
        res_stats.append(symbol_stats)
    # Combine all full symbol stats.
    res_stats_df = pd.concat(res_stats, axis=1).T
    return res_stats_df


def _get_qa_stats_by_year_month(
    data: pd.DataFrame, config: cconconf.Config
) -> pd.DataFrame:
    """
    Get quality assurance stats per full symbol, year, and month.
    """
    #
    data["year"] = data.index.year
    data["month"] = data.index.month
    #
    res_stats = []
    columns_to_groupby = [config["column_names"]["full_symbol"], "year", "month"]
    for index, symbol_data in data.groupby(columns_to_groupby):
        #
        full_symbol, year, month = index
        # Get stats for a full symbol and add them to overall stats.
        symbol_stats = pd.Series(dtype="object", name=full_symbol)
        symbol_stats["year"] = year
        symbol_stats["month"] = month
        symbol_stats["NaNs %"] = 100 * (
            costatis.compute_frac_nan(
                symbol_data[config["column_names"]["close_price"]]
            )
        )
        symbol_stats["volume=0 %"] = 100 * (
            symbol_data[symbol_data["volume"] == 0].shape[0]
            / symbol_data.shape[0]
        )
        symbol_stats["bad data %"] = symbol_stats["NaNs %"] + symbol_stats["volume=0 %"]
        res_stats.append(symbol_stats)
    res_stats_df = pd.concat(res_stats, axis=1).T
    #
    res_stats_df["year"] = res_stats_df["year"].astype(int)
    res_stats_df["month"] = res_stats_df["month"].astype(int)
    # Set index by full symbol, year, and month.
    res_stats_df = res_stats_df.set_index([res_stats_df.index, "year", "month"])
    return res_stats_df


def _plot_bad_data_stats(bad_data_stats: pd.DataFrame) -> None:
    """
    Plot bad data stats per unique full symbol in data.
    """
    full_symbols = bad_data_stats.index.get_level_values(0).unique()
    for full_symbol in full_symbols:
        bad_data_col_name = "bad data %"
        _ = bad_data_stats.loc[full_symbol].plot.bar(
            y=bad_data_col_name, rot=0, title=full_symbol
        )

In [7]:
def set_index_ts(df):
    df["timestamp"] = df["timestamp"].apply(
        lambda x: hdateti.convert_unix_epoch_to_timestamp(x)
    )
    df = df.set_index("timestamp")
    return df

In [8]:
def percentage(df, df_loc):
    result = 100*len(df_loc)/len(df)
    return round(result, 2)

def log_into_exchange(exchange) -> ccxt.Exchange:
    """
    Log into an exchange via CCXT and return the corresponding
    `ccxt.Exchange` object.
    """
    # Select credentials for provided exchange.
    credentials = hsecret.get_secret(exchange)
    # Enable rate limit.
    credentials["rateLimit"] = True
    exchange_class = getattr(ccxt, exchange)
    # Create a CCXT Exchange class object.
    exchange = exchange_class(credentials)
    hdbg.dassert(
        exchange.checkRequiredCredentials(),
        msg="Required credentials not passed",
    )
    return exchange

In [29]:
def load_ccxt_data(currency_pair, since, exchange):
    ccxt_data = exchange.fetch_ohlcv(
            currency_pair,
            timeframe="1m",
            since=since,
            limit=500)
    columns = ["timestamp", "open", "high", "low", "close", "volume"]
    bars = pd.DataFrame(ccxt_data, columns=columns)
    return bars

In [30]:
def get_all_data(exchange, currency_pair, start_timestamp, end_timestamp):
    all_bars = []
    duration = exchange.parse_timeframe("1m") * 100
    for t in range(
            start_timestamp,
            end_timestamp + duration,
            duration * 500,
        ):
        print(t, duration)
        bars = load_ccxt_data(currency_pair, t, ccxt_exchange)
        all_bars.append(bars)
        time.sleep(1)
    return pd.concat(all_bars)

# CcxtHistoricalPqByTileClient

In [9]:
client = icdcl.CcxtHistoricalPqByTileClient(
    config["data"]["version"],
    config["data"]["resample_1min"],
    config["load"]["data_dir"],
    config["data"]["partition_mode"],
    aws_profile=config["load"]["aws_profile"],
)

In [10]:
universe = client.get_universe()
universe

['binance::ADA_USDT',
 'binance::AVAX_USDT',
 'binance::BNB_USDT',
 'binance::BTC_USDT',
 'binance::DOGE_USDT',
 'binance::EOS_USDT',
 'binance::ETH_USDT',
 'binance::LINK_USDT',
 'binance::SOL_USDT',
 'ftx::BNB_USDT',
 'ftx::BTC_USDT',
 'ftx::DOGE_USDT',
 'ftx::ETH_USDT',
 'ftx::LINK_USDT',
 'ftx::SOL_USDT',
 'ftx::XRP_USDT',
 'gateio::ADA_USDT',
 'gateio::AVAX_USDT',
 'gateio::BNB_USDT',
 'gateio::BTC_USDT',
 'gateio::DOGE_USDT',
 'gateio::EOS_USDT',
 'gateio::ETH_USDT',
 'gateio::FIL_USDT',
 'gateio::LINK_USDT',
 'gateio::SOL_USDT',
 'gateio::XRP_USDT',
 'kucoin::ADA_USDT',
 'kucoin::AVAX_USDT',
 'kucoin::BNB_USDT',
 'kucoin::BTC_USDT',
 'kucoin::DOGE_USDT',
 'kucoin::EOS_USDT',
 'kucoin::ETH_USDT',
 'kucoin::FIL_USDT',
 'kucoin::LINK_USDT',
 'kucoin::SOL_USDT',
 'kucoin::XRP_USDT']

# Binance::DOGE_USDT

In [11]:
binance_data = client.read_data(
    ["binance::DOGE_USDT"],
    config["data"]["start_ts"],
    config["data"]["end_ts"],
    config["data"]["columns"],
    config["data"]["filter_data_mode"],
)

,full_symbol,open,high,low,close,volume
2019-10-01 00:00:00+00:00,binance::DOGE_USDT,0.002379,0.00238,0.002379,0.00238,143877.0
,...,...,...,...,...,...
2022-05-06 13:56:00+00:00,binance::DOGE_USDT,0.1241,0.1242,0.124,0.1241,176923.0


,col_name,dtype,num_unique,num_nans,first_elem,type(first_elem)
0,index,"datetime64[ns, UTC]",1488968 / 1488968 = 100.00%,0 / 1488968 = 0.00%,2019-10-01T00:00:00.000000000,<class 'numpy.datetime64'>
1,full_symbol,object,1 / 1488968 = 0.00%,0 / 1488968 = 0.00%,binance::DOGE_USDT,<class 'str'>
2,open,float64,191026 / 1488968 = 12.83%,0 / 1488968 = 0.00%,0.0023791,<class 'numpy.float64'>
3,high,float64,177607 / 1488968 = 11.93%,0 / 1488968 = 0.00%,0.00238,<class 'numpy.float64'>
4,low,float64,178090 / 1488968 = 11.96%,0 / 1488968 = 0.00%,0.0023791,<class 'numpy.float64'>
5,close,float64,189862 / 1488968 = 12.75%,0 / 1488968 = 0.00%,0.00238,<class 'numpy.float64'>
6,volume,float64,834051 / 1488968 = 56.02%,0 / 1488968 = 0.00%,143877.0,<class 'numpy.float64'>


# df=
index=[2019-07-05 12:00:00+00:00, 2022-05-06 13:56:00+00:00]
columns=full_symbol,open,high,low,close,volume
shape=(1488968, 6)
* type=



,full_symbol,open,high,low,close,volume
2019-10-01 00:00:00+00:00,binance::DOGE_USDT,0.002379,0.00238,0.002379,0.00238,143877.0
2019-10-01 00:01:00+00:00,binance::DOGE_USDT,0.00238,0.00238,0.00238,0.00238,0.0
2019-10-01 00:02:00+00:00,binance::DOGE_USDT,0.00238,0.00238,0.00238,0.00238,0.0
,...,...,...,...,...,...
2022-05-06 13:54:00+00:00,binance::DOGE_USDT,0.1241,0.1243,0.124,0.1242,1233386.0
2022-05-06 13:55:00+00:00,binance::DOGE_USDT,0.1243,0.1244,0.1241,0.1242,834687.0
2022-05-06 13:56:00+00:00,binance::DOGE_USDT,0.1241,0.1242,0.124,0.1241,176923.0


,full_symbol,open,high,low,close,volume
2019-07-05 12:00:00+00:00,binance::DOGE_USDT,0.00449,0.0046,0.00376,0.0042,60726008.0
2019-07-05 12:01:00+00:00,binance::DOGE_USDT,0.0042,0.004387,0.0042,0.0043,84307704.0
2019-07-05 12:02:00+00:00,binance::DOGE_USDT,0.0043,0.004475,0.0043,0.004475,48182744.0
,...,...,...,...,...,...
2022-05-06 13:54:00+00:00,binance::DOGE_USDT,0.1241,0.1243,0.124,0.1242,1233386.0
2022-05-06 13:55:00+00:00,binance::DOGE_USDT,0.1243,0.1244,0.1241,0.1242,834687.0
2022-05-06 13:56:00+00:00,binance::DOGE_USDT,0.1241,0.1242,0.124,0.1241,176923.0


,full_symbol,open,high,low,close,volume
2019-07-05 12:00:00+00:00,binance::DOGE_USDT,0.00449,0.0046,0.00376,0.0042,60726008.0
2019-07-05 12:01:00+00:00,binance::DOGE_USDT,0.0042,0.004387,0.0042,0.0043,84307704.0
2019-07-05 12:02:00+00:00,binance::DOGE_USDT,0.0043,0.004475,0.0043,0.004475,48182744.0
,...,...,...,...,...,...
2022-05-06 13:54:00+00:00,binance::DOGE_USDT,0.1241,0.1243,0.124,0.1242,1233386.0
2022-05-06 13:55:00+00:00,binance::DOGE_USDT,0.1243,0.1244,0.1241,0.1242,834687.0
2022-05-06 13:56:00+00:00,binance::DOGE_USDT,0.1241,0.1242,0.124,0.1241,176923.0


In [12]:
binance_2019_09 = binance_data.loc[(binance_data.index.year == 2019) & (binance_data.index.month == 9)]
binance_2019_09_volume_0 = binance_2019_09.loc[binance_2019_09["volume"] == 0]

In [13]:
binance_2019_09

,full_symbol,open,high,low,close,volume
timestamp,,,,,,
2019-09-01 00:00:00+00:00,binance::DOGE_USDT,0.002453,0.002453,0.002453,0.002453,0.0
2019-09-01 00:01:00+00:00,binance::DOGE_USDT,0.002453,0.002453,0.002453,0.002453,0.0
2019-09-01 00:02:00+00:00,binance::DOGE_USDT,0.002452,0.002452,0.002452,0.002452,6927.0
2019-09-01 00:03:00+00:00,binance::DOGE_USDT,0.002452,0.002452,0.002452,0.002452,0.0
2019-09-01 00:04:00+00:00,binance::DOGE_USDT,0.002452,0.002452,0.002452,0.002452,0.0
...,...,...,...,...,...,...
2019-09-30 23:55:00+00:00,binance::DOGE_USDT,0.002375,0.002375,0.002375,0.002375,0.0
2019-09-30 23:56:00+00:00,binance::DOGE_USDT,0.002375,0.002375,0.002375,0.002375,0.0
2019-09-30 23:57:00+00:00,binance::DOGE_USDT,0.002375,0.002375,0.002375,0.002375,0.0


In [14]:
_LOG.info(binance_2019_09_volume_0.shape)
binance_2019_09_volume_0

INFO  (31664, 6)


,full_symbol,open,high,low,close,volume
timestamp,,,,,,
2019-09-01 00:00:00+00:00,binance::DOGE_USDT,0.002453,0.002453,0.002453,0.002453,0.0
2019-09-01 00:01:00+00:00,binance::DOGE_USDT,0.002453,0.002453,0.002453,0.002453,0.0
2019-09-01 00:03:00+00:00,binance::DOGE_USDT,0.002452,0.002452,0.002452,0.002452,0.0
2019-09-01 00:04:00+00:00,binance::DOGE_USDT,0.002452,0.002452,0.002452,0.002452,0.0
2019-09-01 00:06:00+00:00,binance::DOGE_USDT,0.002454,0.002454,0.002454,0.002454,0.0
...,...,...,...,...,...,...
2019-09-30 23:55:00+00:00,binance::DOGE_USDT,0.002375,0.002375,0.002375,0.002375,0.0
2019-09-30 23:56:00+00:00,binance::DOGE_USDT,0.002375,0.002375,0.002375,0.002375,0.0
2019-09-30 23:57:00+00:00,binance::DOGE_USDT,0.002375,0.002375,0.002375,0.002375,0.0


# Extractor

In [10]:
ccxt_binance_DOGE_exchange = imvcdeexcl.CcxtExchange("binance")

In [16]:
sleep_time_in_secs = 1
start_timestamp = pd.Timestamp("2019-09-01")
end_timestamp = pd.Timestamp("2019-09-30")
ccxt_binance_DOGE = ccxt_binance_DOGE_exchange.download_ohlcv_data(
    "DOGE/USDT",
    start_timestamp=start_timestamp,
    end_timestamp=end_timestamp,
    sleep_time_in_secs=sleep_time_in_secs,
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [01:55<00:00,  1.38s/it]


In [17]:
ccxt_binance_DOGE = set_index_ts(ccxt_binance_DOGE)
ccxt_binance_DOGE.loc[ccxt_binance_DOGE['volume'] == 0]

,open,high,low,close,volume,end_download_timestamp
timestamp,,,,,,
2019-09-01 00:00:00+00:00,0.002453,0.002453,0.002453,0.002453,0.0,2022-05-13 17:33:36.618068+00:00
2019-09-01 00:01:00+00:00,0.002453,0.002453,0.002453,0.002453,0.0,2022-05-13 17:33:36.618068+00:00
2019-09-01 00:03:00+00:00,0.002452,0.002452,0.002452,0.002452,0.0,2022-05-13 17:33:36.618068+00:00
2019-09-01 00:04:00+00:00,0.002452,0.002452,0.002452,0.002452,0.0,2022-05-13 17:33:36.618068+00:00
2019-09-01 00:06:00+00:00,0.002454,0.002454,0.002454,0.002454,0.0,2022-05-13 17:33:36.618068+00:00
...,...,...,...,...,...,...
2019-09-30 03:54:00+00:00,0.002165,0.002165,0.002165,0.002165,0.0,2022-05-13 17:35:30.608896+00:00
2019-09-30 03:55:00+00:00,0.002165,0.002165,0.002165,0.002165,0.0,2022-05-13 17:35:30.608896+00:00
2019-09-30 03:56:00+00:00,0.002165,0.002165,0.002165,0.002165,0.0,2022-05-13 17:35:30.608896+00:00


In [18]:
ccxt_binance_DOGE

,open,high,low,close,volume,end_download_timestamp
timestamp,,,,,,
2019-09-01 00:00:00+00:00,0.002453,0.002453,0.002453,0.002453,0.0,2022-05-13 17:33:36.618068+00:00
2019-09-01 00:01:00+00:00,0.002453,0.002453,0.002453,0.002453,0.0,2022-05-13 17:33:36.618068+00:00
2019-09-01 00:02:00+00:00,0.002452,0.002452,0.002452,0.002452,6927.0,2022-05-13 17:33:36.618068+00:00
2019-09-01 00:03:00+00:00,0.002452,0.002452,0.002452,0.002452,0.0,2022-05-13 17:33:36.618068+00:00
2019-09-01 00:04:00+00:00,0.002452,0.002452,0.002452,0.002452,0.0,2022-05-13 17:33:36.618068+00:00
...,...,...,...,...,...,...
2019-09-30 03:55:00+00:00,0.002165,0.002165,0.002165,0.002165,0.0,2022-05-13 17:35:30.608896+00:00
2019-09-30 03:56:00+00:00,0.002165,0.002165,0.002165,0.002165,0.0,2022-05-13 17:35:30.608896+00:00
2019-09-30 03:57:00+00:00,0.002165,0.002165,0.002165,0.002165,31325.0,2022-05-13 17:35:30.608896+00:00


Where`volume = 0`, data from columns `open`, `high`, `low`, `close` is exactly the same from previous row where `volume != 0`. It could mean that `volume = 0` rows are `NaNs` at the source, so it could be the way exchange handles missing data.

In [19]:
print(percentage(ccxt_binance_DOGE, ccxt_binance_DOGE.loc[ccxt_binance_DOGE['volume'] == 0]))

73.92


In [20]:
df = (binance_2019_09 - ccxt_binance_DOGE)
nan = df.loc[df.close.isna()]
_LOG.info(f"{nan.index.min()}, {nan.index.max()}")
nan

INFO  2019-09-30 04:00:00+00:00, 2019-09-30 23:59:00+00:00


,close,end_download_timestamp,full_symbol,high,low,open,volume
timestamp,,,,,,,
2019-09-30 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 04:01:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 04:02:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 04:03:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 04:04:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2019-09-30 23:55:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 23:56:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-30 23:57:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
 pd.set_option("display.float_format", "{:.8f}".format)

# CCXT w/o Extractor

In [28]:
ccxt_exchange = log_into_exchange('binance')

In [33]:
ccxt_df = get_all_data(ccxt_exchange, "DOGE/USDT", 1567296000000, 1569887999000)

1567296000000 6000
1567299000000 6000
1567302000000 6000
1567305000000 6000
1567308000000 6000
1567311000000 6000
1567314000000 6000
1567317000000 6000
1567320000000 6000
1567323000000 6000
1567326000000 6000
1567329000000 6000
1567332000000 6000
1567335000000 6000
1567338000000 6000
1567341000000 6000
1567344000000 6000
1567347000000 6000
1567350000000 6000
1567353000000 6000
1567356000000 6000
1567359000000 6000
1567362000000 6000
1567365000000 6000
1567368000000 6000
1567371000000 6000
1567374000000 6000
1567377000000 6000
1567380000000 6000
1567383000000 6000
1567386000000 6000
1567389000000 6000
1567392000000 6000
1567395000000 6000
1567398000000 6000
1567401000000 6000
1567404000000 6000
1567407000000 6000
1567410000000 6000
1567413000000 6000
1567416000000 6000
1567419000000 6000
1567422000000 6000
1567425000000 6000
1567428000000 6000
1567431000000 6000
1567434000000 6000
1567437000000 6000
1567440000000 6000
1567443000000 6000
1567446000000 6000
1567449000000 6000
156745200000

1568592000000 6000
1568595000000 6000
1568598000000 6000
1568601000000 6000
1568604000000 6000
1568607000000 6000
1568610000000 6000
1568613000000 6000
1568616000000 6000
1568619000000 6000
1568622000000 6000
1568625000000 6000
1568628000000 6000
1568631000000 6000
1568634000000 6000
1568637000000 6000
1568640000000 6000
1568643000000 6000
1568646000000 6000
1568649000000 6000
1568652000000 6000
1568655000000 6000
1568658000000 6000
1568661000000 6000
1568664000000 6000
1568667000000 6000
1568670000000 6000
1568673000000 6000
1568676000000 6000
1568679000000 6000
1568682000000 6000
1568685000000 6000
1568688000000 6000
1568691000000 6000
1568694000000 6000
1568697000000 6000
1568700000000 6000
1568703000000 6000
1568706000000 6000
1568709000000 6000
1568712000000 6000
1568715000000 6000
1568718000000 6000
1568721000000 6000
1568724000000 6000
1568727000000 6000
1568730000000 6000
1568733000000 6000
1568736000000 6000
1568739000000 6000
1568742000000 6000
1568745000000 6000
156874800000

1569888000000 6000


In [34]:
ccxt_df = set_index_ts(ccxt_df)
ccxt_df.index.min(), ccxt_df.index.max(), ccxt_df.shape

(Timestamp('2019-09-01 00:00:00+0000', tz='UTC'),
 Timestamp('2019-10-01 08:19:00+0000', tz='UTC'),
 (432500, 5))

In [35]:
ccxt_df = ccxt_df.loc[ccxt_df.index.month == 9]

In [36]:
ccxt_df.isna().value_counts()

open   high   low    close  volume
False  False  False  False  False     429750
dtype: int64

In [38]:
ccxt_df.loc[ccxt_df['volume'] != 0]

,open,high,low,close,volume
timestamp,,,,,
2019-09-01 00:02:00+00:00,0.00,0.00,0.00,0.00,6927.00
2019-09-01 00:05:00+00:00,0.00,0.00,0.00,0.00,85551.00
2019-09-01 00:08:00+00:00,0.00,0.00,0.00,0.00,4300.00
2019-09-01 00:24:00+00:00,0.00,0.00,0.00,0.00,192594.00
2019-09-01 00:35:00+00:00,0.00,0.00,0.00,0.00,8844.00
...,...,...,...,...,...
2019-09-30 23:38:00+00:00,0.00,0.00,0.00,0.00,6000.00
2019-09-30 23:42:00+00:00,0.00,0.00,0.00,0.00,375.00
2019-09-30 23:44:00+00:00,0.00,0.00,0.00,0.00,8589.00


# Summary


|CCXT | | ||			Extractor	| | | |Client | | |
|------|--|-||-------------|-|-|-|------|-|-|
|date|Number of NaN rows %|	Total number of rows| `volume=0` %	|Number of NaN rows %|	Total number of rows| `volume=0` %| Number of NaN rows %|	Total number of rows| `volume=0` %|
|2019-09|	0          |	                   429750|	      73.22%   	|	0          |	                   42000|	      73.92%   |      0|	             43200| 73.3%|


- The huge amount of data from CCXT is duplicates. Unique values are 43200.
- Data between `2019-09-30 04:00:00+00:00` and `2019-09-30 23:59:00+00:00` is absent (`Extractor`).
-`NaNs` in client data is a resampling.
- Where volume = 0, data from columns open, high, low, close is exactly the same from previous row where volume != 0. It could mean that volume = 0 rows are NaNs at the source, so it could be the way exchange handles missing data.